In [4]:
import time
import torch
import numpy as np
from train import train
from importlib import import_module
from utils import build_dataset, build_iterator, get_time_dif

In [5]:
dataset = '/home/liuyichen/workspace/work/dataset/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt'  
x = import_module('model')
config = x.Config(dataset)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样

In [3]:
start_time = time.time()
print("Loading data...")
train_data, dev_data, test_data = build_dataset(config)
train_iter = build_iterator(train_data, config)
dev_iter = build_iterator(dev_data, config)
test_iter = build_iterator(test_data, config)
time_dif = get_time_dif(start_time)
print("Time usage:", time_dif)

Loading data...


5717it [00:02, 2041.01it/s]

Time usage: 0:00:03


# FinBERT+RCNN

In [4]:
print("Training model...")
model = x.Model(config).to(config.device)
train(config, model, train_iter, dev_iter, test_iter)

Training model...


/home/liuyichen/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch [1/3]
Iter:      0,  Train Loss:  0.93,  Train Acc: 61.72%,  Val Loss:  0.95,  Val Acc: 61.52%,  Time: 0:00:01 *
Epoch [2/3]
Epoch [3/3]
Iter:    100,  Train Loss: 0.092,  Train Acc: 96.88%,  Val Loss:  0.11,  Val Acc: 95.70%,  Time: 0:00:20 *


/home/liuyichen/workspace/work/train.py:100: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config.save_path))


Test Loss:  0.14,  Test Acc: 95.12%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

    positive     0.9600    0.9781    0.9689       319
    negative     0.9590    0.8931    0.9249       131
     neutral     0.8923    0.9355    0.9134        62

    accuracy                         0.9512       512
   macro avg     0.9371    0.9356    0.9357       512
weighted avg     0.9516    0.9512    0.9509       512

Confusion Matrix...
[[312   4   3]
 [ 10 117   4]
 [  3   1  58]]
Time usage: 0:00:00


# ONLY FinBERT

In [6]:
from train import test
from utils import dataset_for_bert, evaluate_pipeline
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
senti_finbert = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path='/home/liuyichen/finbert-tone',num_labels=3)
senti_tokenizer = BertTokenizer.from_pretrained('/home/liuyichen/finbert-tone')
senti_nlp = pipeline("text-classification", model=senti_finbert, tokenizer=senti_tokenizer, device=0)

In [7]:
testdata_for_bert = dataset_for_bert(config)

In [8]:
evaluate_pipeline(senti_nlp, testdata_for_bert)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0.8934755990904321